In [1]:
# Dependencies
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Make results reproducible
#seed = 1235
#np.random.seed(seed)
#tf.set_random_seed(seed)

In [3]:
#Loading the dataset
X = np.load('male_array_1.npy')
data_Y = np.load('male_array_2.npy')

data_Y = pd.DataFrame(data_Y)
data_Y.columns = ['혈당', '수축기혈압', '이완기혈압', '총콜레스테롤','HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백']
#Y = pd.get_dummies(data_Y['혈당'])
#y = np.array(Y)


In [4]:
label = '혈당'

X = pd.DataFrame(X)
y = data_Y[label]

In [5]:

X[label] = y
n = len(X[X[label] == 3])
temp = pd.concat([X[X[label] == 1][:n], X[X[label] == 2][:n], X[X[label] == 3][:n]], axis = 0)
y = pd.get_dummies(temp[label])
y = np.array(y)
X = np.array(temp[[0,1,2,3,4]])
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [6]:
X

array([[10.        , 10.94408733, 10.44055703, 10.13198481,  3.        ],
       [12.        , 10.94408733, 10.00639853, 10.02204825,  2.        ],
       [ 9.        , 10.19802596,  8.70392301,  7.7133806 ,  1.        ],
       ...,
       [14.        , 10.94408733, 10.87471553, 10.90154069,  2.        ],
       [ 9.        , 10.19802596, 10.87471553, 10.57173103,  3.        ],
       [16.        , 10.19802596,  8.70392301,  8.37299993,  1.        ]])

In [7]:
# Creating a Train and a Test Dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [8]:
# Session
#sess = tf.Session()

In [9]:
# Interval / Epochs
interval = 100
epoch = 1500

In [10]:
# Initialize placeholders
with tf.name_scope("input") as scope:
    X_data = tf.placeholder(shape=[None, 5], dtype=tf.float32)

with tf.name_scope("target") as scope:
    y_target = tf.placeholder(shape=[None, 3], dtype=tf.float32)

In [11]:
input_layer_nodes = 5
hidden_layer_nodes1 = 100
hidden_layer_nodes2 = 30
hidden_layer_nodes3 = 30
output_layer_nodes =  3


In [12]:
import keras

keep_prob = tf.placeholder("float")

with tf.name_scope("weight1") as scope:
    w1 = tf.get_variable("w1", shape=[input_layer_nodes, hidden_layer_nodes1], initializer=tf.keras.initializers.he_normal())
with tf.name_scope("bias1")as scope:
    b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes1]))   # First Bias

with tf.name_scope("weight2") as scope:
    w2 = tf.get_variable("w2", shape=[hidden_layer_nodes1, hidden_layer_nodes2], initializer=tf.keras.initializers.he_normal())
with tf.name_scope("bias2") as scope:
    b2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes2]))   # Second Bias

with tf.name_scope("weight3") as scope:
    w3 = tf.get_variable("w3", shape=[hidden_layer_nodes2, hidden_layer_nodes3], initializer=tf.keras.initializers.he_normal())
with tf.name_scope("bias3") as scope:
    b3 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes3]))

with tf.name_scope("weight4") as scope:
    w4 = tf.get_variable("w4", shape=[hidden_layer_nodes3, output_layer_nodes], initializer=tf.keras.initializers.he_normal())
with tf.name_scope("bias4") as scope:
    b4 = tf.Variable(tf.random_normal(shape=[output_layer_nodes]))


Using TensorFlow backend.


In [13]:
# Operations

with tf.name_scope("layer1") as scope:
    hidden_output1 = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
    hidden_output1 = tf.nn.dropout(hidden_output1, keep_prob = keep_prob)

with tf.name_scope("layer2") as scope:
    hidden_output2 = tf.nn.relu(tf.add(tf.matmul(hidden_output1, w2), b2))
    hidden_output2 = tf.nn.dropout(hidden_output2, keep_prob = keep_prob)

with tf.name_scope("layer3") as scope: 
    hidden_output3 = tf.nn.relu(tf.add(tf.matmul(hidden_output2, w3), b3))
    hidden_output3 = tf.nn.dropout(hidden_output3, keep_prob = keep_prob)

    
logits = tf.add(tf.matmul(hidden_output3, w4), b4)

with tf.name_scope("output_layer") as scope:
    final_output = tf.nn.softmax(logits)


In [14]:
#Cost
with tf.name_scope("cost") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_target))
    tf.summary.scalar("loss",loss)

In [15]:
# Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [16]:
# Initialize variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
#with tf.Session() as sess:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("tensorboard/", sess.graph)

In [18]:
# Training
print('Training the model...')
for i in range(1, (epoch + 1)):
#    sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train, keep_prob : 0.5})
#    if i % 10 == 0:
#        summary = sess.run(merged, feed_dict = {X_data : X_train, y_target: y_train, keep_prob: 0.5})
#        writer.add_summary(summary,i)
    if i % interval == 0:
        print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train, keep_prob
                                                                  : 0.5}))
    _, summary = sess.run([optimizer, merged], feed_dict = {X_data : X_train, y_target: y_train, keep_prob: 0.5})
    writer.add_summary(summary,i)

Training the model...
('Epoch', 100, '|', 'Loss:', 1.0855206)
('Epoch', 200, '|', 'Loss:', 1.0801681)
('Epoch', 300, '|', 'Loss:', 1.0785232)
('Epoch', 400, '|', 'Loss:', 1.0779972)
('Epoch', 500, '|', 'Loss:', 1.0771735)
('Epoch', 600, '|', 'Loss:', 1.0763856)
('Epoch', 700, '|', 'Loss:', 1.0762298)
('Epoch', 800, '|', 'Loss:', 1.0751703)
('Epoch', 900, '|', 'Loss:', 1.0735583)
('Epoch', 1000, '|', 'Loss:', 1.0741247)
('Epoch', 1100, '|', 'Loss:', 1.0726273)
('Epoch', 1200, '|', 'Loss:', 1.0731963)
('Epoch', 1300, '|', 'Loss:', 1.0745302)
('Epoch', 1400, '|', 'Loss:', 1.0728371)
('Epoch', 1500, '|', 'Loss:', 1.0737394)


In [19]:
# Prediction
#print()
#for i in range(len(X_test)):
#    print('Actual:', y_test[i], 'Predicted:', (sess.run(final_output, feed_dict={X_data: [X_test[i]], keep_prob : 1.0})))

In [20]:
count = 0
pred = []
for i in range(len(X_test)):
    if np.argmax(y_test[i]) == np.argmax(sess.run(final_output, feed_dict={X_data: [X_test[i]], keep_prob:1.0})):
        count += 1
    pred.append(np.argmax(sess.run(final_output, feed_dict={X_data: [X_test[i]], keep_prob:1.0})))
print('accuracy : ', float(count)/len(X_test))

('accuracy : ', 0.46090105910113593)


In [21]:
pred2 = pd.get_dummies(pred)
pred2 = np.array(pred2)

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred2))

             precision    recall  f1-score   support

          0       0.56      0.45      0.50     12114
          1       0.37      0.08      0.13     12128
          2       0.43      0.86      0.57     12204

avg / total       0.45      0.46      0.40     36446

